In [ ]:
!pip install qiskit 
!pip install qiskit-aer
!pip install pylatexenc

# Import necessary utils

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import XGate, ZGate

import os, sys
sys.path.append(os.getcwd())

In [ ]:
# This for Google Colab, so that we can import the files from the repository
!git clone https://github.com/algolab-quantique/IMACA-CMAI-Grover.git

sys.path.insert(0,'/content/IMACA-CMAI-Grover')

In [ ]:
# Import the utils functions from the repository
from utils import get_disjunction_control_state, get_disjunction_qubits

# Write the seven statements of the Pincus problem

A clause of the form 

$
    (x_0 \lor x_2 \lor \neg x_3)
$

is written : 
```
clause = {'x0' : True, 'x2' : True, 'x3' : False}
````

Remember : 

* x0 = Fearful
* x1 = Happy
* x2 = Unhealthy
* x3 = Noisy


In [ ]:
clauses = []

# If a native is healthy and courageous, he is also noisy
clauses.append({'x2' : True, 'x0' : True, 'x3' : True})

# If a native is quiet and fearful, he is also happy
# (¬x0 ⌵ x3 ⌵ x1)  
# write the definition of the clause here

# If a native is healthy and noisy, he is also happy
# (x2 ⌵ ¬x3 ⌵ x1)
# write the definition of the clause here

# If a native is happy and quite, he is also healthy
# (¬x1 ⌵ x3 ⌵ ¬x2) 
# write the definition of the clause here

# If a native is fearful and healthy, he is also sad
# (¬x0 ⌵ x2 ⌵ ¬x1) 
# write the definition of the clause here

# If a native is sad and unhealthy, he is also fearful
# (x1 ⌵ ¬x2 ⌵ x0)  
# write the definition of the clause here

# If a native is fearful and noisy, he is also healthy
# (¬x0 ⌵ ¬x3 ⌵ ¬x2) 
# write the definition of the clause here

# Setup known data of the problem

In [ ]:
# x0 : fearful, x1 : happy, x2 : unhealthy, x3 : noisy
nb_variables = 4
# 7 statements
nb_clauses = len(clauses)

# Create function to transform disjunction statement into gate

Take a disjunction statement and transform it into quantum circuit. 

For example, the statement : 

$
    (x_0 \lor x_1 \lor \neg x_2)
$

would result in the following gates : 

<img src="disj_circuit.png" alt= “” width="15%" height="15%">

Each disjunction is transformed into a circuit containing as many qubits as there is variables in the statement,
with one ancillar qubit. The circuit is then converted to a gate that can be appended to the main grover circuit,
by specifying the right control and ancillar qubits.

In [ ]:
# Tranform logical disjunction into Quantum Gate
def logical_disjunction_to_gate(disj_clause):
    # Get the number of variables included in the disjuntion clause
    nb_disj_variables = len(disj_clause)
    
    # Set the number of qubits as the number of variables in disjunction + 1 ancillar qubit
    nb_qubits = nb_disj_variables + 1
    disj_qc = QuantumCircuit(nb_qubits)
    qubits = disj_qc.qubits
    
    # Get the right control state
    ctrl_state = get_disjunction_control_state(disj_clause)

    # Add an X gate to the ancillar qubit (the last qubit. Position -1 gives the last element)
    disj_qc.x(qubits[-1])
    
    # Create a multi-controlled X gate with specified control state. It has as many control
    # qubits as there are variables in the disjunction. 
    mc_xgate = XGate().control(num_ctrl_qubits=nb_disj_variables, ctrl_state = ctrl_state)
    disj_qc.append(mc_xgate, qubits)
    
    # Transform the disjunction circuit into a gate to append to the grover circuit
    disj_gate = disj_qc.to_gate(label='mcx')
    
    return disj_gate

Let's see how this function works.

In [ ]:
clause_test = {'x0' : True, 'x1' : True, 'x2' : False}

test_qc = QuantumCircuit(4)
disj_gate = logical_disjunction_to_gate(clause_test)
test_qc.append(disj_gate,range(4))
test_qc.decompose().draw('mpl')

# Construct the Oracle

In [ ]:
# Create Quantum Registers for the variables and the clauses
var_qubits = QuantumRegister(nb_variables, name='x')
clause_qubits = QuantumRegister(nb_clauses, name='c')

In [ ]:
# Setup the clause circuit
clauses_circuit = QuantumCircuit(var_qubits, clause_qubits)

# Add each clause as a disjunction gate : 

# Clause 0
# Convert the first clause into a quantum gate
gate = logical_disjunction_to_gate(clauses[0])
# Get the qubits associated with the first clause, including the variables involved in the clause.
c_qubits = get_disjunction_qubits(clauses[0], clause_qubits[0], var_qubits)
# Append the gate to the oracle circuit using the right qubits
clauses_circuit.append(gate, c_qubits)

# Clause 1
# Write your code to append a gate associated to the second clause here #

# Clause 2
# Write your code to append a gate associated to the clause here #

# ...

# Clause 6
# Write your code to append a gate associated to the clause here #


# P.S : you can use for loops if you want.  

# Show the circuit : 
clauses_circuit.decompose(gates_to_decompose=['mcx'], reps=2).draw(output='mpl')

In [ ]:
# Setup the oracle circuit
oracle_circuit = QuantumCircuit(var_qubits, clause_qubits)

# Add clauses circuit 
# --- transform to gate to use label (only for drawing purposes) ---
oracle_circuit.append(clauses_circuit.to_gate(label='clauses_circuit'), clauses_circuit.qubits)

# Add multi-control Z gate 
mc_z_gate = ZGate().control(nb_clauses - 1)
oracle_circuit.append(mc_z_gate, clause_qubits)

# Add reversed clauses_circuit
oracle_circuit.append(clauses_circuit.reverse_ops().to_gate(label='clauses_circuit'), oracle_circuit.qubits)

# Show circuit
oracle_circuit.decompose(gates_to_decompose=['clauses_circuit', 'mcx'], reps=2).draw(output='mpl')


# Construct the diffuser

Complete the diffuser circuit.

In [ ]:
# Setup the grover circuit
diffuser_circuit = QuantumCircuit(var_qubits)

# Add H and X gates (change of basis)

# Add multi-control Z gate 
mc_z_gate = ZGate().control(nb_variables - 1)
diffuser_circuit.append(mc_z_gate, var_qubits)

# Add X and H gates (revert change of basis)

# Show circuit
diffuser_circuit.draw(output='mpl')

# Assemble Grover circuit

Complete theGrover circuit.

In [ ]:
# Setup the Grover circuit
c_bits = ClassicalRegister(nb_variables)
grover_circuit = QuantumCircuit(var_qubits, clause_qubits, c_bits)

# Apply a H gate for all variables qubits
### Add 4 H gates on the right qubits (variables qubits)

# Setup the number of iterations
nb_iterations = 1 # Play with the number of iterations

# Add as many oracle and diffusor circuits as there are iterations
for it in range(nb_iterations):
    grover_circuit.append(oracle_circuit.to_gate(label='oracle'), grover_circuit.qubits)
    grover_circuit.barrier(grover_circuit.qubits)
    grover_circuit.append(diffuser_circuit.to_gate(label='diffusor'), grover_circuit.qubits[0:nb_variables])
    
# Add measurement to evaluate the circuit
grover_circuit.measure(var_qubits, c_bits)

# Show circuit
grover_circuit.decompose(gates_to_decompose=['oracle', 'clauses_circuit', 'diffusor', 'mcx'], reps=3).draw(output='mpl',
                                                                                                   scale=0.8)

# Measure solution

In [ ]:
# Prepare a simulation to run and measure the solution
def run_circuit(circ: QuantumCircuit) -> dict:
    """
    Run a quantum circuit on the AerSimulator and return the counts
    @param circ: QuantumCircuit to run
    @return: dictionary of measurement results and their counts
    """
    simulator = AerSimulator()
    circ = transpile(circ, simulator)
    result = simulator.run(circ, shots=1000).result()
    return result.get_counts(circ)

In [ ]:
# Execute circuit and get counts
counts = run_circuit(grover_circuit)
print(counts)

In [ ]:
plot_histogram(counts)

Remember : 

* x0 = Fearful
* x1 = Happy
* x2 = Unhealthy
* x3 = Noisy
